# 💡 MechOnFluid 本体加载与推理演示手册

> 本文档旨在系统性展示如何基于 MechOn-Fluid 本体进行 **OWL 本体的加载、推理与可视化分析**。  
> 适用于教学演示、模型验证与垂直领域知识工程的入门实践。

你将学习以下内容：
1. 创建，使用聚合文件
2. owl full与owl dl的区别与桥接方法
3. 不同owl本体集间的桥接
4. 推理测试

---

🧑‍💻 文档作者：庞嘉顺  
📅 编写日期：2025 年 7 月  
📘 项目背景：本 notebook 属于 [MechOn] 与 [SessionSync] 项目的基础模块示范之一

本体内容：MechOn-Fluid

<div align="center">
    <img src="../docs/sessync.png" width="700" alt="sessync logo" />
</div>

In [1]:
import subprocess, tempfile, os
from rdflib import Graph

ttl_path = "./examples/2/mechon-fluid.owl"

# 1) Turtle -> RDF/XML 临时文件
g = Graph().parse(ttl_path, format="turtle")
tmp_xml = tempfile.mkstemp(suffix=".owl")[1]
g.serialize(tmp_xml, format="xml")


<Graph identifier=N9362734a2acb48b7ba1c42fd3331ee1a (<class 'rdflib.graph.Graph'>)>

# 📚 1. 聚合文件的创建与引用（Ontology Aggregation via `owl:imports`）

在本节中，我们将学习如何通过 **聚合文件（aggregate ontology）** 管理多个模块化本体，实现统一引用、自动加载、推理共享等目标。

我们以简化版本的 **MechOn-Fluid** 本体集为例进行演示，说明如何使用 `owl:imports` 构建本体的入口文件。

---

## 🧩 1.1 本体集架构总览

以下是 MechOn-Fluid 项目的模块化结构图，其中每个子模块（如 `mat.ttl`、`phy.ttl`）都是一个独立维护的领域子本体，而 `mechon-fluid.owl` 是顶层聚合文件（也称为 *entrypoint ontology*）。

<div align="center">
    <img src="../docs/core.png" width="700" alt="mechon-fluid structure" />
</div>

---

## 📦 1.2 聚合文件 mechon-fluid.owl 的作用

文件 `mechon-fluid.owl` 作为整个本体系统的统一入口，其核心任务是：

- 使用 `owl:imports` 引用所有子模块；
- 定义统一的命名空间（base URI）；
- 提供版本信息，供推理机和开发工具识别。

一旦加载该文件，所有子本体将自动被导入，无需逐一加载。

---

## 🧰 1.3 聚合文件结构解析

下面是 `mechon-fluid.owl` 的典型结构，采用标准 OWL/RDF Turtle 格式：

```ttl
@prefix :     <https://purl.mechon.org/ont/fluid#> .
@prefix owl:  <http://www.w3.org/2002/07/owl#> .
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml:  <http://www.w3.org/XML/1998/namespace> .
@prefix xsd:  <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

@base <https://purl.mechon.org/ont/fluid#> .

<https://purl.mechon.org/ont/fluid>
    rdf:type owl:Ontology ;
    owl:versionIRI <https://purl.mechon.org/ont/fluid/0.1.0> ;
    owl:imports 
        <https://purl.mechon.org/ont/calc/0.1.0> ,
        <https://purl.mechon.org/ont/geo/0.1.0> ,
        <https://purl.mechon.org/ont/mat/0.1.0> ,
        <https://purl.mechon.org/ont/phy/0.1.0> .
```
---

上边的owl语句包含三个部分，
### 1. 命名空间与基URI: @prefix / @base
这里的 <https://purl.mechon.org/ont/fluid#> 是该本体的命名空间 IRI，我们可以简称为：

Namespace IRI = 用于构造类、属性、个体等名称的“前缀地址”。

例如：
```ttl
:StressTensor a :TensorField .
```
会被展开为：
```ttl
<https://purl.mechon.org/ont/fluid#StressTensor>
```
---
### 2. 本体资源的标识与元数据定义：rdf:type owl:Ontology
使用 rdf:type owl:Ontology 声明该文件是一个 OWL 本体，并为其指定统一的本体 URI 与版本 URI，便于工具识别和版本控制。
```ttl
<https://purl.mechon.org/ont/fluid>
    rdf:type owl:Ontology ;
    owl:versionIRI <https://purl.mechon.org/ont/fluid/0.1.0> .
```
|字段          |	含义          |
|------------|-------------------|
|rdf:type owl:Ontology	|声明该资源是一个 OWL 本体|
|<.../ont/fluid>	|本体的全局唯一标识符（Ontology IRI）|
|owl:versionIRI	|明确指定当前版本的 URI，有助于版本管理与引用一致性|

> ⚠️  注意：本体 URI（Ontology IRI）与命名空间 URI 并不相同，前者指向“整个本体资源”，后者用于构造实体名称。
---

### 3. 加载子模块 `owl:imports`
聚合文件的关键作用是通过 `owl:imports` 导入其他子模块本体，实现模块化开发、分工维护、统一推理。
```ttl
<https://purl.mechon.org/ont/fluid>
    owl:imports 
        <https://purl.mechon.org/ont/calc/0.1.0> ,
        <https://purl.mechon.org/ont/geo/0.1.0> ,
        <https://purl.mechon.org/ont/mat/0.1.0> ,
        <https://purl.mechon.org/ont/phy/0.1.0> .
```
每一个被导入的模块都应该：

- 是一个合法的 OWL 本体

- 在自身文件中使用 rdf:type owl:Ontology 显式声明

- 定义自己的命名空间与 @base URI

- （可选）使用 owl:versionIRI 声明版本

子模块的本体资源标识示例`mat.core.ttl`
> <https://purl.mechon.org/ont/mat> rdf:type owl:Ontology ;
> 
>                                   owl:versionIRI <https://purl.mechon.org/ont/mat/0.1.0> .

In [2]:
import subprocess, tempfile, os
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS, OWL

def check_imports():
    """检查mechon-fluid.owl文件是否成功导入了所有聚合的内容"""
    
    # 加载主文件
    ttl_path = "./examples/2/mechon-fluid.owl"
    g = Graph().parse(ttl_path, format="turtle")
    
    print("=== 检查 mechon-fluid.owl 导入情况 ===\n")
    
    # 1. 检查声明的导入
    print("1. 声明的导入 (owl:imports):")
    ontology_uri = "https://purl.mechon.org/ont/fluid"
    for s, p, o in g.triples((ontology_uri, OWL.imports, None)):
        print(f"   - {o}")
    
    print("\n2. 检查各个核心文件的内容是否被包含:")
    
    # 2. 检查各个核心文件的内容
    core_files = {
        "calc": "./examples/2/calc.core.ttl",
        "geo": "./examples/2/geo.core.ttl", 
        "mat": "./examples/2/mat.core.ttl",
        "phy": "./examples/2/phy.core.ttl"
    }
    
    for name, file_path in core_files.items():
        print(f"\n   --- {name.upper()} 核心文件检查 ---")
        
        # 加载核心文件
        core_g = Graph().parse(file_path, format="turtle")
        
        # 统计核心文件中的类、属性等
        core_classes = list(core_g.subjects(RDF.type, OWL.Class))
        core_obj_props = list(core_g.subjects(RDF.type, OWL.ObjectProperty))
        core_data_props = list(core_g.subjects(RDF.type, OWL.DatatypeProperty))
        
        print(f"   核心文件包含:")
        print(f"   - 类: {len(core_classes)} 个")
        print(f"   - 对象属性: {len(core_obj_props)} 个")
        print(f"   - 数据属性: {len(core_data_props)} 个")
        
        # 检查主文件中是否包含这些内容
        found_classes = 0
        found_obj_props = 0
        found_data_props = 0
        
        for cls in core_classes:
            if (cls, RDF.type, OWL.Class) in g:
                found_classes += 1
        
        for prop in core_obj_props:
            if (prop, RDF.type, OWL.ObjectProperty) in g:
                found_obj_props += 1
                
        for prop in core_data_props:
            if (prop, RDF.type, OWL.DatatypeProperty) in g:
                found_data_props += 1
        
        print(f"   主文件中找到:")
        print(f"   - 类: {found_classes}/{len(core_classes)} 个")
        print(f"   - 对象属性: {found_obj_props}/{len(core_obj_props)} 个")
        print(f"   - 数据属性: {found_data_props}/{len(core_data_props)} 个")
        
        # 计算导入比例
        if len(core_classes) > 0:
            class_ratio = found_classes / len(core_classes) * 100
            print(f"   - 类导入比例: {class_ratio:.1f}%")
        
        if len(core_obj_props) > 0:
            obj_prop_ratio = found_obj_props / len(core_obj_props) * 100
            print(f"   - 对象属性导入比例: {obj_prop_ratio:.1f}%")
            
        if len(core_data_props) > 0:
            data_prop_ratio = found_data_props / len(core_data_props) * 100
            print(f"   - 数据属性导入比例: {data_prop_ratio:.1f}%")
    
    # 3. 检查主文件的总统计
    print(f"\n3. 主文件 (mechon-fluid.owl) 总统计:")
    main_classes = list(g.subjects(RDF.type, OWL.Class))
    main_obj_props = list(g.subjects(RDF.type, OWL.ObjectProperty))
    main_data_props = list(g.subjects(RDF.type, OWL.DatatypeProperty))
    
    print(f"   - 总类数: {len(main_classes)}")
    print(f"   - 总对象属性数: {len(main_obj_props)}")
    print(f"   - 总数据属性数: {len(main_data_props)}")
    
    # 4. 检查是否有QUDT相关的导入
    print(f"\n4. QUDT相关检查:")
    qudt_entities = [s for s, p, o in g.triples((None, None, None)) 
                    if "qudt.org" in str(s) or "qudt.org" in str(o)]
    print(f"   - QUDT相关实体数: {len(qudt_entities)}")
    
    # 5. 检查命名空间
    print(f"\n5. 命名空间检查:")
    namespaces = g.namespaces()
    for prefix, uri in namespaces:
        print(f"   - {prefix}: {uri}")

if __name__ == "__main__":
    check_imports()

=== 检查 mechon-fluid.owl 导入情况 ===

1. 声明的导入 (owl:imports):

2. 检查各个核心文件的内容是否被包含:

   --- CALC 核心文件检查 ---
   核心文件包含:
   - 类: 15 个
   - 对象属性: 1 个
   - 数据属性: 0 个
   主文件中找到:
   - 类: 0/15 个
   - 对象属性: 0/1 个
   - 数据属性: 0/0 个
   - 类导入比例: 0.0%
   - 对象属性导入比例: 0.0%

   --- GEO 核心文件检查 ---
   核心文件包含:
   - 类: 29 个
   - 对象属性: 4 个
   - 数据属性: 0 个
   主文件中找到:
   - 类: 0/29 个
   - 对象属性: 0/4 个
   - 数据属性: 0/0 个
   - 类导入比例: 0.0%
   - 对象属性导入比例: 0.0%

   --- MAT 核心文件检查 ---
   核心文件包含:
   - 类: 39 个
   - 对象属性: 2 个
   - 数据属性: 1 个
   主文件中找到:
   - 类: 0/39 个
   - 对象属性: 0/2 个
   - 数据属性: 0/1 个
   - 类导入比例: 0.0%
   - 对象属性导入比例: 0.0%
   - 数据属性导入比例: 0.0%

   --- PHY 核心文件检查 ---
   核心文件包含:
   - 类: 4 个
   - 对象属性: 4 个
   - 数据属性: 2 个
   主文件中找到:
   - 类: 0/4 个
   - 对象属性: 0/4 个
   - 数据属性: 0/2 个
   - 类导入比例: 0.0%
   - 对象属性导入比例: 0.0%
   - 数据属性导入比例: 0.0%

3. 主文件 (mechon-fluid.owl) 总统计:
   - 总类数: 0
   - 总对象属性数: 0
   - 总数据属性数: 0

4. QUDT相关检查:
   - QUDT相关实体数: 2

5. 命名空间检查:
   - brick: https://brickschema.org/schema/Brick#
   - csvw: http:/